In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q unsloth

In [ ]:
!pip uninstall -y trl unsloth unsloth-zoo peft transformers bitsandbytes
!pip install -q unsloth
!pip install -q datasets evaluate rouge-score bert-score

Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.0 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.9.0+cu126
    Uninstalling torchaudio-2.9.0+cu126:
      Successfully uninstalled torchaudio-2.9.0+cu126


In [ ]:
!pip install -q datasets evaluate rouge-score bert-score



In [ ]:
import torch, random, numpy as np
from datasets import load_dataset
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
import evaluate

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
raw = load_dataset("cnn_dailymail", "3.0.0")

def format_example(example):
    length = random.choice(["short", "medium", "long"])
    return {
        "text": f"""### Instruction:
Summarize the article below in a {length} summary.

### Article:
{example['article'][:3000]}

### Summary:
{example['highlights']}"""
    }

train_dataset = (
    raw["train"]
    .shuffle(seed=42)
    .select(range(1000))
    .map(format_example, remove_columns=raw["train"].column_names)
)

val_dataset = (
    raw["validation"]
    .shuffle(seed=42)
    .select(range(200))
    .map(format_example, remove_columns=raw["validation"].column_names)
)

README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-7B-Instruct-v0.2",
    max_seq_length = 1024, # Increased max_seq_length to match SFTTrainer
    dtype = torch.float16,
    load_in_4bit = True,
)

==((====))==  Unsloth 2026.1.2: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules = [
        "q_proj", "k_proj", "v_proj",
        "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,
    num_train_epochs = 3,
    learning_rate = 2e-4,
    fp16 = True,
    logging_steps = 10,
    save_strategy = "epoch",
    optim = "adamw_8bit",
    report_to = "none",
    seed = 42,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = 1024,
    args = training_args,
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 3 | Total steps = 189
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.805400
20,1.744500
30,1.707600
40,1.683900
50,1.695800
60,1.722500
70,1.568900
80,1.467500
90,1.430200
100,1.447200


TrainOutput(global_step=189, training_loss=1.454659628489661, metrics={'train_runtime': 6691.9471, 'train_samples_per_second': 0.448, 'train_steps_per_second': 0.028, 'total_flos': 1.0745993703412531e+17, 'train_loss': 1.454659628489661, 'epoch': 3.0})

In [ ]:
adapter_path = "/content/mistral-slm-summary"
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)


('/content/mistral-slm-summary/tokenizer_config.json',
 '/content/mistral-slm-summary/special_tokens_map.json',
 '/content/mistral-slm-summary/chat_template.jinja',
 '/content/mistral-slm-summary/tokenizer.model',
 '/content/mistral-slm-summary/added_tokens.json',
 '/content/mistral-slm-summary/tokenizer.json')

In [ ]:
import shutil

zip_path = "/content/mistral-slm-summary.zip"
shutil.make_archive(
    base_name=zip_path.replace(".zip", ""),
    format="zip",
    root_dir=adapter_path
)

zip_path


'/content/mistral-slm-summary.zip'

In [ ]:
from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:


# --------------------------------------------
# Config
# --------------------------------------------
MODEL_DIR = "/content/mistral-slm-summary"
EXAMPLE_ID = 25          # any index you like
SUMMARY_LENGTH = "medium"

# --------------------------------------------
# Load model & tokenizer
# --------------------------------------------
model = AutoPeftModelForCausalLM.from_pretrained(
    MODEL_DIR,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
tokenizer.pad_token = tokenizer.eos_token
model.eval()

# --------------------------------------------
# Generation config
# --------------------------------------------
GEN_CONFIG = GenerationConfig(
    max_new_tokens=90,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id,
)

# --------------------------------------------
# Summarization function
# --------------------------------------------
def summarize(article: str) -> str:
    prompt = f"""### Instruction:
Summarize the article below in a {SUMMARY_LENGTH} summary.

### Article:
{article}

### Summary:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            generation_config=GEN_CONFIG
        )

    decoded = tokenizer.decode(
        output[0],
        skip_special_tokens=True
    )

    # Strip prompt
    return decoded.split("### Summary:")[-1].strip()

# --------------------------------------------
# Load one dataset example
# --------------------------------------------
dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation")

article = dataset[EXAMPLE_ID]["article"]
reference = dataset[EXAMPLE_ID]["highlights"]

# --------------------------------------------
# Run inference
# --------------------------------------------
prediction = summarize(article)

# --------------------------------------------
# Print sanity check outputs
# --------------------------------------------
print("=" * 100)
print("ARTICLE (truncated):\n")
print(article[:900] + "...\n")
print("-" * 100)
print("REFERENCE SUMMARY:\n")
print(reference + "\n")
print("-" * 100)
print("MODEL SUMMARY:\n")
print(prediction)
print("=" * 100)

# --------------------------------------------
# ROUGE evaluation (single example)
# --------------------------------------------
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeLsum"],
    use_stemmer=True
)

rouge_scores = scorer.score(reference, prediction)

print("\nROUGE SCORES (single example):")
for k, v in rouge_scores.items():
    print(f"{k.upper():10s} | P: {v.precision:.4f} | R: {v.recall:.4f} | F1: {v.fmeasure:.4f}")

# --------------------------------------------
# BERTScore evaluation
# --------------------------------------------
P, R, F1 = score(
    [prediction],
    [reference],
    lang="en",
    verbose=False
)

print("\nBERTScore:")
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1:        {F1.mean().item():.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attri

ARTICLE (truncated):

(Rolling Stone)Kanye West fans are used to seeing thousand-dollar price tags attached to the rapper's Air Yeezy sneakers when they're put up for sale on eBay, but the auction site is now being overrun by sellers offering up plastic bags full of air allegedly captured at West's concerts. The gag started Friday when one ambitious seller attempted to sell a Zipperseal bag with "Air From Kanye Show," with 90 bidders driving the price tag to over $60,000, the Telegraph reports. While that auction was ultimately pulled from the auction site, eBay has since been flooded with similar auctions promising air bagged from a Yeezus show, even though the copycats lacked certificates of authenticity proving the air -- likely carbon dioxide -- was grabbed at a West concert. eBay has apparently stopped trying to swat down the new "bagged air" auctions, so buyer beware at this point. Rolling Stone: Kanye ...

-------------------------------------------------------------------------

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore:
Precision: 0.8702
Recall:    0.8942
F1:        0.8820
